In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
from input_pipeline import dataset_tfrecord_pipeline

2024-04-27 08:42:28.666738: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-27 08:42:28.718941: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-27 08:42:28.718971: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-27 08:42:28.719918: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-27 08:42:28.726540: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-27 08:42:28.726945: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
def fast_energy(images):
    n = images.shape[0]
    shifts = [
        (1, 1),  # Right
        (-1, 1),  # Left
        (-1, 0),  # Up
        (1, 0),  # Down
        (-1, 1),  # Up-Right
        (1, 1),  # Down-Right
        (-1, -1),  # Up-Left
        (1, -1),  # Down-Left
    ]
    neighbor_sum = np.zeros_like(images)
    for shift in shifts:
        shifted_mat = np.roll(images, shift=shift, axis=(1, 2))
        neighbor_sum += shifted_mat
    
    return - np.sum(np.multiply(neighbor_sum, images)) / n

In [5]:
energy_temp = []
for temp in np.arange(2.0, 3.1, 0.1):
    data_path = f"../../GetData/Python/Data/Data{temp:.2}.tfrecord"
    dataset = dataset_tfrecord_pipeline(data_path, flatten=False, batch_size=50000)
    data = next(iter(dataset))
    energy = fast_energy(2 * data - 1)
    energy_temp.append((energy, temp))
    print(f"Temp: {temp:.2} Energy: {energy:.3}")

Temp: 2.0 Energy: -7.02e+03
Temp: 2.1 Energy: -6.61e+03
Temp: 2.2 Energy: -6.04e+03
Temp: 2.3 Energy: -5.17e+03
Temp: 2.4 Energy: -4.34e+03
Temp: 2.5 Energy: -3.82e+03
Temp: 2.6 Energy: -3.44e+03
Temp: 2.7 Energy: -3.14e+03
Temp: 2.8 Energy: -2.88e+03
Temp: 2.9 Energy: -2.67e+03
Temp: 3.0 Energy: -2.48e+03


In [9]:
np.savetxt("energy_temp.csv", np.array(energy_temp), delimiter=',', header='Energy,Temp', fmt='%.1f', comments='')
